# Calculate 310-340K and 500-900hPa mean and azimuthal mean absolute vorticity

In [1]:
import os
import sys
sys.path.append('../..')
import glob
import time
import xarray
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib as mpl
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cmocean
import cimf as c
from ddxddy import ddxND, ddyND
%load_ext autoreload
%autoreload 2

In [2]:
p_ref = 100000
cp = 1004
Rdry = 286.9
kappa = Rdry/cp

In [3]:
from numba import float64, guvectorize

@guvectorize(
    "(float32[:], float32[:], float32[:], float32[:])",
    " (n), (n), (m) -> (m)",
    nopython=True,
)
def interp1d_gu(f, x, xi, out):
    """Interpolate field f(x) to xi in ln(x) coordinates."""
    i, imax, x0, f0 = 0, len(xi), x[0], f[0]
    while xi[i]<x0 and i < imax:
        out[i] = np.nan      
        i = i + 1 
    for x1,f1 in zip(x[1:], f[1:]):
        while xi[i] <= x1 and i < imax:
            out[i] = (f1-f0)/np.log(x1/x0)*np.log(xi[i]/x0)+f0
            i = i + 1
        x0, f0 = x1, f1
    while i < imax:
        out[i] = np.nan
        i = i + 1


def xr_interp(data, p, newp):
    print(f"xr_interp: {data.name}")
    interped = xarray.apply_ufunc(
        interp1d_gu,  # first the function
        data,  # now arguments in the order expected by 'interp1_np'
        p,  # as above
        newp,  # as above
        input_core_dims=[['hybrid'], ['hybrid'], ['p']],  # list with one entry per arg
        output_core_dims=[['p']],  # returned data has one dimension
        exclude_dims=set(('hybrid',)),  # dimensions allowed to change size. Must be a set!
        dask="parallelized",
        output_dtypes=[
            data.dtype
        ],  # one per output; could also be float or np.dtype("float64")
    ).persist()
    interped['p'] = newp

    return interped

### Potential temperature coordinates

In [4]:
ds1 = xarray.open_mfdataset(
    '/Users/jasperdejong/Documents/PhD/Irma/Data/LambertGrid/629x989interped/fc2017090512+???.nc',
    combine='nested', concat_dim='valid_time', decode_timedelta=True)

In [5]:
ds1['eta'] = ddxND(ds1.v) - ddyND(ds1.u); print("eta done")
ds1 = c.crop(ds1, d=80); print("crop done")
ds1 = c.convert_wind(ds1); print("wind done")
ds1['u_rad'] = ds1.u_rad.astype('float32')
ds1['v_tan'] = ds1.v_tan.astype('float32')
r, angle = c.distance(ds1.latitude.sel(dy=0,dx=0), ds1.longitude.sel(dy=0,dx=0), 
                      ds1.latitude, ds1.longitude)
ds1 = ds1.assign_coords({'r':r})

eta done
12:29:36: crop()
crop done
12:30:20: convert_wind()
12:30:20: toPolar()
12:30:20: distance()
12:30:20: translational_velocity()
12:30:20: transform_vector()
wind done
12:30:24: distance()


In [6]:
ds1.sel(theta=slice(310,340),dy=0,dx=slice(-50,50)).mean('theta').to_netcdf("figdata_310-340K_avg.nc")

In [7]:
dsm1,_ = c.azimean(ds1)

12:30:24: azimean()
12:30:24: toPolar()
12:30:24: distance()
12:30:24: convert_wind()
12:30:24: translational_velocity()
12:30:25: transform_vector()
12:30:28: azimean_gufunc()
12:30:32: removing ['dy', 'dx'] from azimuthal mean dataset


In [8]:
dsm1.mean('valid_time').to_netcdf("figdata_azimean_theta.nc")

### Pressure coordinates

In [9]:
%%time
ds2 = xarray.open_mfdataset(
    '/Users/jasperdejong/Documents/PhD/Irma/Data/LambertGrid/629x989/fc2017090512+???.nc',
    combine='nested', concat_dim='valid_time', decode_timedelta=True)
ds2['pres'] = (ds2.a + ds2.b * ds2.p0m).astype('float32'); print("pressure done")
ds2['eta'] = ddxND(ds2.v) - ddyND(ds2.u); print("eta done")
ds2['theta'] = (ds2.t  * (p_ref/ds2.pres)**kappa).astype('float32'); print("theta done")
ds2 = c.crop(ds2, d=80); print("crop done")
ds2 = c.convert_wind(ds2); print("wind done")
ds2['u_rad'] = ds2.u_rad.astype('float32')
ds2['v_tan'] = ds2.v_tan.astype('float32')
pi = np.arange(25,1025,25,dtype=ds2.pres.dtype)*100
ds2['eta'] = xr_interp(ds2.eta, ds2.pres, pi)
ds2['theta'] = xr_interp(ds2.theta, ds2.pres, pi)
ds2['u_rad'] = xr_interp(ds2.u_rad, ds2.pres, pi)
ds2['v_tan'] = xr_interp(ds2.v_tan, ds2.pres, pi)
r, angle = c.distance(ds2.latitude.sel(dy=0,dx=0), ds2.longitude.sel(dy=0,dx=0), 
                      ds2.latitude, ds2.longitude)
ds2 = ds2.assign_coords({'r':r})
data = ds2[[var for var in ds2.variables if ('hybrid' not in ds2[var].dims)]]

pressure done
eta done
theta done
12:32:31: crop()
crop done
12:33:40: convert_wind()
12:33:40: toPolar()
12:33:40: distance()
12:33:40: translational_velocity()
12:33:42: transform_vector()
wind done
xr_interp: eta
xr_interp: theta
xr_interp: u_rad
xr_interp: v_tan
12:33:54: distance()
CPU times: user 3min 22s, sys: 3min 41s, total: 7min 3s
Wall time: 3min 21s


In [10]:
data.sel(p=slice(50000,90000),dy=0, dx=slice(-50,50)).mean('p').to_netcdf("figdata_900-500hpa_avg.nc")

In [11]:
dsm2,_ = c.azimean(data)

12:33:54: azimean()
12:33:54: toPolar()
12:33:54: distance()
12:33:54: convert_wind()
12:33:54: convert_wind(): using existing u_rad and v_tan
12:33:54: azimean_gufunc()
12:33:56: removing ['dy', 'dx'] from azimuthal mean dataset


In [12]:
dsm2.mean("valid_time").to_netcdf("figdata_azimean_pres.nc")